In [10]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasRegressor
from skopt import BayesSearchCV

from tensorflow.keras.regularizers import l2
from utils.util import is_laptop
from utils.model import mean_absolute_percentage_error
from utils.data import get_normalized_data
import pandas as pd, numpy as np, tensorflow as tf, os

In [2]:
train, val, test = get_normalized_data()

X_train, y_train = train
X_val, y_val = val
X_test, y_test = test

In [15]:
def build_search_space():
    search_space = {
                    'model__n_layers': np.arange(4,5),# 9),
                    'model__n_units': [64],#, 128, 256, 512],
                    'model__activation': ['relu'],#, 'elu', 'softmax'],
                    'model__penalty': [0],#, 0.01, 0.1, 1],
                    'model__optimizer': ['adam']#, 'rmsprop', 'adamax']
                }
    
    return search_space

In [13]:
# Define hyperparameter grid
param_grid = {
    'n_estimators': [50],#, 100, 150],
    'max_depth': [None],#, 10, 20],
    'min_samples_split': [2]#, 5, 10]
}

In [5]:
def build_model(n_layers, n_units, activation, penalty, optimizer):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty), input_shape = (X_train.shape[1], )))
    model.add(tf.keras.layers.BatchNormalization())

    for _ in range(n_layers):
        model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
        model.add(tf.keras.layers.BatchNormalization())

    model.add(tf.keras.layers.Dense(units = n_units, activation = activation, kernel_regularizer=l2(penalty)))
    model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(units = 1, activation = 'linear'))

    model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])

    return model

In [18]:
model = KerasRegressor(build_fn = build_model, epochs = 10, batch_size = 64)
search_space = build_search_space()

kfold = KFold(n_splits = 2, shuffle=True, random_state=42)
scoring = make_scorer(accuracy_score)

grid_search = GridSearchCV(estimator=model, param_grid=search_space, scoring = scoring, cv = kfold, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


ValueError: 
All the 2 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\scikeras\wrappers.py", line 735, in fit
    self._fit(
  File "c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-packages\scikeras\wrappers.py", line 900, in _fit
    self._fit_keras_model(
  File "c:\Users\Brynjar Geir\Documents\lokaverkefni_vel\lokaverkefni_vel\code\.venv\Lib\site-pack

In [ ]:
hparams

In [ ]:
grid_search.save

In [ ]:
# Evaluate the model
mape = model.evaluate(X_test, y_test)
print(f'Model evaluates to: {mape}%')

In [ ]:
from datetime import date
model.save(f'./saved_models/nn-{date.today()}.keras')